In [1]:
def initialize_board():
    board = [['EMPTY' for _ in range(8)] for _ in range(8)]

    # 중앙 4칸에 초기 돌 배치
    board[3][3] = 'WHITE'
    board[3][4] = 'BLACK'
    board[4][3] = 'BLACK'
    board[4][4] = 'WHITE'

    return board

In [2]:
BLACK_SYMBOL = '●'   # U+25CF
WHITE_SYMBOL = '○'   # U+25CB
EMPTY_SYMBOL = '·'   # U+00B7

def print_board(board):
    col_labels = "  A B C D E F G H"
    print(col_labels)
    for i, row in enumerate(board):
        row_str = f"{i + 1} " + " ".join(
            EMPTY_SYMBOL if cell == 'EMPTY' else
            (BLACK_SYMBOL if cell == 'BLACK' else WHITE_SYMBOL)
            for cell in row
        )
        print(row_str)

In [3]:
# === 좌표 변환 함수 ===
def to_internal_coords(move_str):
    col_str, row_str = move_str[0], move_str[1]
    x = int(row_str) - 1
    y = ord(col_str.upper()) - ord('A')
    return x, y

def to_display_coords(x, y):
    col = chr(ord('A') + y)
    row = str(x + 1)
    return col + row

In [4]:
DIRECTIONS = [(-1, -1), (-1, 0), (-1, 1),
              (0, -1),          (0, 1),
              (1, -1),  (1, 0), (1, 1)]

def get_opponent(player):
    return 'BLACK' if player == 'WHITE' else 'WHITE'

def is_on_board(x, y):
    return 0 <= x < 8 and 0 <= y < 8

def is_valid_move(board, player, x, y):
    if board[x][y] != 'EMPTY':
        return False

    opponent = get_opponent(player)
    
    for dx, dy in DIRECTIONS:
        nx, ny = x + dx, y + dy
        found_opponent = False

        while is_on_board(nx, ny) and board[nx][ny] == opponent:
            nx += dx
            ny += dy
            found_opponent = True

        if found_opponent and is_on_board(nx, ny) and board[nx][ny] == player:
            return True

    return False

def get_valid_moves(board, player):
    moves = []
    for x in range(8):
        for y in range(8):
            if is_valid_move(board, player, x, y):
                moves.append((x, y))
    return moves

In [5]:
def apply_move(board, player, x, y):
    if not is_valid_move(board, player, x, y):
        return False  # Invalid move

    board[x][y] = player
    opponent = get_opponent(player)

    for dx, dy in DIRECTIONS:
        nx, ny = x + dx, y + dy
        path = []

        # 상대 돌 계속 탐색
        while is_on_board(nx, ny) and board[nx][ny] == opponent:
            path.append((nx, ny))
            nx += dx
            ny += dy

        # 자기 돌로 끝나면 뒤집기
        if is_on_board(nx, ny) and board[nx][ny] == player:
            for px, py in path:
                board[px][py] = player

    return True  # Move applied successfully

In [6]:
def play_game(policy_black, policy_white, black_name="BLACK", white_name="WHITE", verbose=True):
    board = initialize_board()
    current_player = 'BLACK'

    while True:
        valid_moves = get_valid_moves(board, current_player)

        if not valid_moves:
            opponent = get_opponent(current_player)
            if not get_valid_moves(board, opponent):
                break
            if verbose:
                print(f"{current_player} has no valid moves. Skipping turn.")
            current_player = opponent
            continue

        if verbose:
            print_board(board)
            print(f"\n{black_name if current_player == 'BLACK' else white_name}'s turn ({current_player})")

        if current_player == 'BLACK':
            move = policy_black(board, current_player, valid_moves)
        else:
            move = policy_white(board, current_player, valid_moves)

        if move not in valid_moves:
            print(f"Invalid move: {move}. Game aborted.")
            return None  # 실패 처리

        apply_move(board, current_player, move[0], move[1])
        current_player = get_opponent(current_player)

    # 게임 종료
    black_count = sum(row.count('BLACK') for row in board)
    white_count = sum(row.count('WHITE') for row in board)

    if verbose:
        print_board(board)
        print(f"\nGame Over! {black_name}: {black_count}, {white_name}: {white_count}")

    # 승자 반환
    if black_count > white_count:
        return black_name
    elif white_count > black_count:
        return white_name
    else:
        return 'DRAW'

In [7]:
def greedy_policy(board, player, valid_moves):
    max_flip = -1
    best_move = None

    for x, y in valid_moves:
        temp_board = [row.copy() for row in board]
        apply_move(temp_board, player, x, y)
        flipped = sum(row.count(player) for row in temp_board) - sum(row.count(player) for row in board)

        if flipped > max_flip:
            max_flip = flipped
            best_move = (x, y)

    return best_move

In [8]:
def manual_policy(board, player, valid_moves):
    display_moves = [to_display_coords(x, y) for (x, y) in valid_moves]
    print("Valid moves:", display_moves)

    while True:
        move_str = input("Enter your move (e.g., D3): ").strip().upper()
        if len(move_str) != 2:
            print("Invalid format. Use column(A-H) + row(1-8).")
            continue
        try:
            x, y = to_internal_coords(move_str)
            if (x, y) in valid_moves:
                return (x, y)
        except:
            pass
        print("Invalid move. Try again.")

In [9]:
def evaluate_matchup(policy1, policy2, name1="AI1", name2="AI2", n_games=50):
    results = {name1: 0, name2: 0, 'DRAW': 0}

    for i in range(n_games):
        winner = play_game(policy1, policy2, black_name=name1, white_name=name2, verbose=False)
        results[winner] += 1

    print(f"\nMatch Results ({n_games} games):")
    print(f"{name1} Wins: {results[name1]}")
    print(f"{name2} Wins: {results[name2]}")
    print(f"DRAWs     : {results['DRAW']}")

In [10]:
import time
import copy

# 내부 숫자 표현
EMPTY, BLACK, WHITE = 0, 1, -1

# 변환 딕셔너리
symbol_to_num = {'EMPTY': EMPTY, 'BLACK': BLACK, 'WHITE': WHITE}

def opponent(color):
    return -color

def is_valid_move(board, row, col, color):
    if board[row][col] != EMPTY:
        return False
    directions = [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]
    for dr, dc in directions:
        r, c = row + dr, col + dc
        count = 0
        while 0 <= r < 8 and 0 <= c < 8 and board[r][c] == opponent(color):
            r += dr
            c += dc
            count += 1
        if count > 0 and 0 <= r < 8 and 0 <= c < 8 and board[r][c] == color:
            return True
    return False

def get_valid_moves(board, color):
    return [(r, c) for r in range(8) for c in range(8) if is_valid_move(board, r, c, color)]

def make_move(board, row, col, color):
    new_board = copy.deepcopy(board)
    new_board[row][col] = color
    directions = [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]
    for dr, dc in directions:
        r, c = row + dr, col + dc
        flips = []
        while 0 <= r < 8 and 0 <= c < 8 and new_board[r][c] == opponent(color):
            flips.append((r, c))
            r += dr
            c += dc
        if 0 <= r < 8 and 0 <= c < 8 and new_board[r][c] == color:
            for fr, fc in flips:
                new_board[fr][fc] = color
    return new_board

# 가중치 기반 평가 함수
POSITION_WEIGHTS = [
    [100, -20, 10, 5, 5, 10, -20, 100],
    [-20, -50, -2, -2, -2, -2, -50, -20],
    [10, -2, 0, 0, 0, 0, -2, 10],
    [5, -2, 0, 0, 0, 0, -2, 5],
    [5, -2, 0, 0, 0, 0, -2, 5],
    [10, -2, 0, 0, 0, 0, -2, 10],
    [-20, -50, -2, -2, -2, -2, -50, -20],
    [100, -20, 10, 5, 5, 10, -20, 100],
]

def evaluate_strong(board, color):
    score = 0
    my_moves = get_valid_moves(board, color)
    opp_moves = get_valid_moves(board, opponent(color))
    mobility = len(my_moves) - len(opp_moves)
    for r in range(8):
        for c in range(8):
            if board[r][c] == color:
                score += POSITION_WEIGHTS[r][c]
            elif board[r][c] == opponent(color):
                score -= POSITION_WEIGHTS[r][c]
    corners = [(0,0), (0,7), (7,0), (7,7)]
    for r, c in corners:
        if board[r][c] == color:
            score += 100
        elif board[r][c] == opponent(color):
            score -= 100
    x_squares = [(0,1), (1,0), (1,1), (0,6), (1,6), (1,7), (6,0), (6,1), (7,1), (6,6), (6,7), (7,6)]
    for r, c in x_squares:
        if board[r][c] == color:
            score -= 30
        elif board[r][c] == opponent(color):
            score += 30
    score += 5 * mobility
    return score

def minimax(board, depth, alpha, beta, maximizing, color, start_time, eval_func):
    """
    시간 제한(9.5초)에 걸려도 지금까지 찾은 best_move 를 꼭 반환하도록 수정한 버전
    """
    # ──────────────────────────────────────────────────────────────────────────
    # 1) 탐색 깊이 바닥이거나 시간 초과 시 → 평가점수와 current best_move 반환
    # ──────────────────────────────────────────────────────────────────────────
    if depth == 0 or time.time() - start_time > 9.5:
        # 이 시점까지 best_move 가 없을 수도 있으므로 None 허용
        return eval_func(board, color), None

    valid_moves = get_valid_moves(board, color if maximizing else opponent(color))
    if not valid_moves:
        return eval_func(board, color), None

    best_move = None  # 지금까지 발견한 최선의 수

    # ──────────────────────────────────────────────────────────────────────────
    # 2) maximizing / minimizing 분기
    # ──────────────────────────────────────────────────────────────────────────
    if maximizing:
        max_eval = float('-inf')
        for move in valid_moves:
            # 시간 초과 체크 ― 루프 내부에서도 계속 확인
            if time.time() - start_time > 9.5:
                break

            new_board = make_move(board, move[0], move[1], color)
            eval_score, _ = minimax(
                new_board, depth - 1, alpha, beta, False,
                color, start_time, eval_func
            )

            if eval_score > max_eval:
                max_eval = eval_score
                best_move = move

            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break  # 가지치기
        return max_eval, best_move

    else:  # minimizing
        min_eval = float('inf')
        for move in valid_moves:
            if time.time() - start_time > 9.5:
                break

            new_board = make_move(board, move[0], move[1], opponent(color))
            eval_score, _ = minimax(
                new_board, depth - 1, alpha, beta, True,
                color, start_time, eval_func
            )

            if eval_score < min_eval:
                min_eval = eval_score
                best_move = move

            beta = min(beta, eval_score)
            if beta <= alpha:
                break
        return min_eval, best_move

# 🔥 최종 policy 함수 (네 프레임워크용)
def min_policy(board, player, valid_moves):
    # board: [['EMPTY', 'BLACK', ...]] 형식
    internal_board = [[symbol_to_num[cell] for cell in row] for row in board]
    my_color = 1 if player == 'BLACK' else -1
    start_time = time.time()
    moves_played = sum(cell != 0 for row in internal_board for cell in row)
    depth = 6 if moves_played < 50 else 8
    _, move = minimax(internal_board, depth, float('-inf'), float('inf'), True, my_color, start_time, evaluate_strong)
    return move  # (x, y)


In [11]:
def random_policy(board, player, valid_moves):
    import random
    return random.choice(valid_moves)

In [12]:
evaluate_matchup(greedy_policy, greedy_policy, name1="AI1", name2="AI2", n_games=10)


Match Results (10 games):
AI1 Wins: 0
AI2 Wins: 0
DRAWs     : 10


In [ ]:
winner = play_game(greedy_policy, manual_policy, black_name="GreedyAI", white_name="You")
print("Winner:", winner)